# 隐语义模型
协同过滤算法的特点就是完全没有利用到物品本身或者是用户自身的属性，仅仅利用了用户与物品的交互信息就可以实现推荐，是一个可解释性很强，非常直观的模型，但是也存在一些问题，其中一个就是处理稀疏矩阵的能力比较弱， 所以为了使得协同过滤更好处理稀疏矩阵问题， 增强泛化能力， 从协同过滤中衍生出矩阵分解模型（Matrix Factorization，MF）或者叫隐语义模型，就是在协同过滤共现矩阵的基础上，使用更稠密的隐向量表示用户和物品，挖掘用户和物品的隐含兴趣和隐含特征，在一定程度上弥补协同过滤模型处理稀疏矩阵能力不足的问题

隐语义模型最早在文本领域被提出，用于找到文本的隐含语义。在 2006 年， 被用于推荐中，它的核心思想是通过隐含特征（latent factor）联系用户兴趣和物品，基于用户的行为找出潜在的主题和分类，然后对物品进行自动聚类，划分到不同类别/主题（用户的兴趣）

**优点**

- **泛化能力强**。在一定程度上解决了数据稀疏的问题
- **空间复杂度低**。不需要存储协同过滤模型服务阶段所需的庞大的用户相似性或物品相似性矩阵，只需存储用户和物品隐向量。空间复杂度由 $n^2$ 级别降低到 $(n + m)k$ 级别
- **更好的扩展性和灵活性**。矩阵分解的最终产出是用户和物品隐向量，这其实与深度学习中的 Embedding 思想不谋而合，因此矩阵分解的结果也非常便于与其他特征进行组合和拼接，并便于与深度学习网络进行无缝结合

**缺点**

与协同过滤一样，矩阵分解同样不方便加入用户、物品和上下文相关的特征，这使得矩阵分解丧失了利用很多有效信息的机会，同时在缺乏用户历史行为时，无法进行有效的推荐

## FunkSVD
2006 年的 Netflix Prize 之后，Simon Funk 公布了一个矩阵分解算法叫做 FunkSVD，后来被 Netflix Prize 的冠军 Koren 称为 Latent Factor Model（LFM）。在隐语义模型中，我们使用同样维度的隐向量 $\pmb{p}$、$\pmb{q}$ 来分别表示用户和物品，然后再用用户的隐向量与物品的隐向量的点积来表示用户对该物品的偏好程度 $\hat{r}_{u,i} = \pmb{p}_u^\top \pmb{q}_i = \sum_{k=1}^K p_{u,k} q_{i,k}$

<img src="../../img/lfm.png" width="500">

其中 $\pmb{p}_u$ 为用户 $u$ 的隐向量，$\pmb{q}_i$ 为物品 $i$ 的隐向量，$K$ 为向量的维度

**目标函数**

$$
\begin{aligned}
J
=& \sum_{(u,i) \in \mathcal{S}} (r_{u,i} - \hat{r}_{u,i})^2 + \lambda \left\| \pmb{p}_u \right\|^2 + \lambda \left\| \pmb{q}_i \right\|^2 \\
=& \sum_{(u,i) \in \mathcal{S}} (r_{u,i} - \sum_{k=1}^K p_{u,k} q_{i,k})^2 + \lambda \left\| \pmb{p}_u \right\|^2 + \lambda \left\| \pmb{q}_i \right\|^2
\end{aligned}
$$

$$(\pmb{p}^{*},\pmb{q}^{*}) = \arg \min \limits_{\pmb{p},\pmb{q}} J$$

其中 $r_{u,i}$ 为用户 $u$ 对物品 $i$ 兴趣度的真实值，$\hat{r}_{u,i}$ 为用户 $u$ 对物品 $i$ 兴趣度的预测值，$\mathcal{S}$ 为用户-物品集，$\lambda \left\| \pmb{p}_u \right\|^2 + \lambda \left\| \pmb{q}_i \right\|^2$ 是用来防止过拟合的正则化项

**梯度下降**

求导

$$\frac{\partial J}{\partial p_{u,k}} = -2 (r_{u,i} - \sum_{k=1}^K p_{u,k} q_{i,k}) q_{i,k} + 2 \lambda p_{u,k}$$

$$\frac{\partial J}{\partial q_{i,k}} = -2 (r_{u,i} - \sum_{k=1}^K p_{u,k} q_{i,k}) p_{u,k} + 2 \lambda q_{i,k}$$

令 $e_{u,i} = r_{u,i} - \sum_{k=1}^K p_{u,k} q_{i,k}$，得

$$p_{u,k} = p_{u,k} + \alpha (e_{u,i} q_{i,k} - \lambda p_{u,k})$$

$$q_{i,k} = q_{i,k} + \alpha (e_{u,i} p_{u,k} - \lambda q_{i,k})$$

## BiasSVD
实际情况下，一个评分系统有些固有的属性和用户物品无关，而用户也有些属性和物品无关，物品也有些属性和用户无关。 因此，Netfix Prize 中提出了另一种 LFM，在原来的基础上加了偏置项，来消除用户和物品打分的偏差，其预测公式如下：

$$\hat{r}_{u,i} = \mu + b_u + b_i + \pmb{p}_u^\top \pmb{q}_i$$

其中 $\mu$，$b_u$，$b_i$ 的含义如下

- $\mu$：训练集中所有记录的评分的全局平均数，作为全局偏差常量。在不同网站中，因为网站定位和销售物品不同，网站的整体评分分布也会显示差异。比如有的网站中用户就喜欢打高分，有的网站中用户就喜欢打低分。而全局平均数可以表示网站本身对用户评分的影响

- $b_u$：用户偏差系数，可以使用用户 $u$ 给出的所有评分的均值，也可以当作训练参数。这一项表示了用户的评分习惯中和物品没有关系的因素。比如有些用户比较苛刻，对什么东西要求很高，那么他评分就会偏低，而有些用户比较宽容，对什么东西都觉得不错，那么评分就偏高

- $b_i$：物品偏差系数，可以使用物品 $i$ 收到的所有评分的均值，也可以当作训练参数。这一项表示了物品接受的评分中和用户没有关系的因素。比如有些物品本身质量就很高，因此获得的评分相对比较高，有的物品本身质量很差，因此获得的评分相对较低

引入偏置项后得到新的目标函数

$$J = \sum_{(u,i) \in \mathcal{S}} (r_{u,i} - \mu - b_u - b_i - \sum_{k=1}^K p_{u,k} q_{i,k})^2 + \lambda (\left\| \pmb{p}_u \right\|^2 + \left\| \pmb{q}_i \right\|^2 + b_u^2 + b_i^2)$$

求导

$$\frac{\partial J}{\partial b_u} = -2 (r_{u,i} - \mu - b_u - b_i - \sum_{k=1}^K p_{u,k} q_{i,k}) + 2 \lambda b_u$$

$$\frac{\partial J}{\partial b_i} = -2 (r_{u,i} - \mu - b_u - b_i - \sum_{k=1}^K p_{u,k} q_{i,k}) + 2 \lambda b_i$$

令 $e_{u,i} = r_{u,i} - \mu - b_u - b_i - \sum_{k=1}^K p_{u,k} q_{i,k}$，得

$$b_u = b_u + \alpha (e_{u,i} - \lambda b_u)$$

$$b_i = b_i + \alpha (e_{u,i} - \lambda b_i)$$

## 超参数选择

- 向量的维度 $K$
- 学习率 $\alpha$
- 正则化参数 $\lambda$
- 正负样本比 $ratio$

## 显性反馈数据

In [1]:
import numpy as np
import pandas as pd

In [2]:
'''
@输入参数
R：M*N的评分矩阵
K：隐特征向量维度
max_iter：最大迭代次数
alpha：学习率
lamda：正则化系数

@输出
分解之后的P、Q
P：初始化用户特征矩阵 M*K
Q：初始化物品特征矩阵 N*K
'''
# 核心算法
def LFM(R, K, max_iter, alpha=0.0002, lamda=0.002):
    # 基本维度参数定义
    M = len(R)
    N = len(R[0])
    
    # P、Q初始值，随机生成
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    
    # 偏置项
    mu = np.mean(R)
    bu = np.random.randn(M, 1)
    bi = np.random.randn(N, 1)
    
    # 训练
    for step in range(max_iter):
        # 遍历所有样本 (u,i)
        for u in range(M):
            for i in range(N):
                eui = R[u][i] - np.dot(P[u], Q[i]) - bu[u] - bi[i] - mu

                # 带入公式，按照梯度下降算法更新当前的Pu与Qi
                P[u] += alpha * (eui * Q[i] - lamda * P[u])
                Q[i] += alpha * (eui * P[u] - lamda * Q[i])
                bu[u] += alpha * (eui - lamda * bu[u])
                bi[i] += alpha * (eui - lamda * bi[i])
    
    return P, Q, bu, bi, mu

In [3]:
# 评分矩阵R
R = np.array([[4, 0, 2, 0, 1],
              [0, 0, 2, 3, 1],
              [4, 1, 2, 0, 1],
              [4, 1, 2, 5, 1],
              [3, 0, 5, 0, 2],
              [1, 0, 3, 0, 4]])
#给定超参数
K= 5
max_iter = 5000
alpha = 0.0002
lamda = 0.004

P, Q, bu, bi, mu = LFM(R, K, max_iter, alpha, lamda)

In [4]:
abs(P @ Q.T + bu + bi.T + mu - R)

array([[0.01430647, 0.16016227, 0.08446286, 0.04528794, 0.13418465],
       [0.36144479, 0.23910096, 0.19788685, 0.0265354 , 0.2579777 ],
       [0.05984533, 0.27486016, 0.05486614, 0.04110086, 0.09972324],
       [0.22768592, 0.15217326, 0.11137953, 0.03325899, 0.14078125],
       [0.10568611, 0.02433118, 0.2659384 , 0.05372061, 0.20915985],
       [0.1157556 , 0.17606944, 0.15833171, 0.02611558, 0.23291415]])

## 隐性反馈数据
LFM 在显性反馈数据（也就是评分数据）上解决评分预测问题并达到了很好的精度，而对于隐性反馈数据集合，这种数据集的特点是只有正样本（用户喜欢什么物品），而没有负样本（用户对什么物品不感兴趣）

**注意**
- 对每个用户，要保证正负样本的平衡（数目相似）
- 对每个用户采负样本时，要选取那些很热门，而用户却没有行为的物品
- 对于用户-物品集 $\mathcal{S} = \{(u,i)\}$，如果 $(u,i)$ 是正样本，则 $R_{u,i} = 1$，否则 $R_{u,i} = 0$

In [5]:
good_ratings = pd.read_csv('../data/ratings/good_ratings.csv')
good_ratings

,user_id,item_id,rating
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
79995,943,1067,1
79996,943,1074,1
79997,943,1188,1
79998,943,1228,1


In [6]:
# 负样本选取：选取热门但用户却没有行为的物品作为负样本
def select_negative_samples(df, user_id, ratio=1):
    positive_samples = list(set(df[df['user_id']==user_id]['item_id'].values)) # 用户喜欢的物品
    series = df['item_id'].value_counts()
    negative_samples = list(series.index)
    negative_samples = [item for item in negative_samples if item not in positive_samples] # 用户没有评分的物品
    num_ns = len(positive_samples) * ratio
    return negative_samples[:num_ns]

In [7]:
user_ids = list(set(good_ratings['user_id'].values))
bad_ratings = None
for user_id in user_ids:
    ns = select_negative_samples(good_ratings, user_id)
    if len(ns) > 0:
        d = {'user_id': user_id, 'item_id': ns, 'rating': 0}
        tmp = pd.DataFrame(data=d)
        if bad_ratings is None:
            bad_ratings = tmp
        else:
            bad_ratings = pd.concat([bad_ratings, tmp], axis=0, ignore_index=True)

In [8]:
ratings = pd.concat([good_ratings, bad_ratings], axis=0, ignore_index=True)
ratings

,user_id,item_id,rating
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
159995,943,109,0
159996,943,528,0
159997,943,705,0
159998,943,129,0


In [9]:
ratings.to_csv('../data/ratings/ratings.csv', index=False)

In [10]:
dataset = ratings.values

In [11]:
def predict(P, Q, u, i):
    p = P[u]
    q = Q[i]
    r = p.dot(q)
    r = 1.0 / (1 + np.exp(-r))
    return r

In [12]:
def LFM_v2(dataset, K=5, max_iter=5, alpha=0.001, lamda=0.002):
    
    M = dataset[:, 0].max() + 1 # 用户数
    N = dataset[:, 1].max() + 1 # 物品数
    
    # P、Q初始值，随机生成
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    
    # 训练
    for step in range(max_iter):
        # 遍历所有样本 (u,i)
        for j in range(len(dataset)):
            u, i, rui = dataset[j, 0], dataset[j, 1], dataset[j, 2]
            eui = rui - predict(P, Q, u, i)
            # 带入公式，按照梯度下降算法更新当前的Pu与Qi
            P[u] += alpha * (eui * Q[i] - lamda * P[u])
            Q[i] += alpha * (eui * P[u] - lamda * Q[i])
        alpha *= 0.9

    return P, Q

In [13]:
P, Q = LFM_v2(dataset)

In [14]:
def recommend(df, user_id, P, Q, TopN=10):
    positive_samples = list(set(df[df['user_id']==user_id]['item_id'].values)) # 用户喜欢的物品
    all_items = list(set(df['item_id'].values)) # 所有物品
    no_rating_items = [item for item in all_items if item not in positive_samples] # 用户没有评分的物品
    predictions = [predict(P, Q, user_id, item_id) for item_id in no_rating_items]
    series = pd.Series(predictions, index=no_rating_items)
    series = series.sort_values(ascending=False)[:TopN]
    return series

In [15]:
recommend(good_ratings, 2, P, Q)

1304    0.965544
593     0.962734
1435    0.959387
1313    0.957346
336     0.956541
1535    0.956230
1273    0.956089
348     0.953780
1356    0.953434
841     0.952475
dtype: float64